In [1]:
import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from graphviz import Source, render
import pydot

import datetime
import time

import sys
sys.path.append('py_modules') # below modules are in this folder
#print(sys.path)
import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm

In [2]:
# [Essential] specify input feeder data for IEEE 123-node test feeder

# Enter the path/name of the impedance model data (excel file)
filepath = "feeder_impedance_models/"
modelpath = filepath + "003_GB_IEEE37.xlsx" # accurate wrt public 123NF data
#modelpath = filepath + "003_GB_IEEE37_zeroCrossZ.xlsx" # accurate wrt public 123NF data

#file_name = string specifying name of dot file created when make_graph() is called
file_name = '37NF'
#file_name = 'demo123NF.dot'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4800
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_799'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

2022-06-03 15:08:44.494702


In [3]:
# [ESSENTIAL] create feeder object

#fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
fin_feeder = ff.feeder_init(modelpath,'','',timesteps,Vbase_ll,Sbase,depths,leaves)

print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
print(list(node_index_map.items())[:10]) # print first 10 elements of node_index_map

R,X=hm.createRXmatrices_3ph(fin_feeder, depths,file_name)
n=round(len(R)/3)
print('n=',n)

# #print('depths=',depths) # check this is populated, lists how far each node is from substation
# #print('depths length=',len(depths))



Finished initializing feeder
[('bus_701', 0), ('bus_702', 1), ('bus_703', 2), ('bus_704', 3), ('bus_705', 4), ('bus_706', 5), ('bus_707', 6), ('bus_708', 7), ('bus_709', 8), ('bus_710', 9)]
n= 35
n= 35


In [ ]:
print(node_index_map)

In [ ]:
# Save R and X matrices to csv to import into matlab
# np.savetxt reference: https://thispointer.com/how-to-save-numpy-array-to-a-csv-file-using-numpy-savetxt-in-python/
np.savetxt('Rmat_37NF_mod.csv', R, delimiter=',')
np.savetxt('Xmat_37NF_mod.csv', X, delimiter=',')

In [ ]:
# write busnames into a csv
import csv
graphNodes_noSub=hm.remove_subst_nodes(fin_feeder, file_name) # remove substation nodes, busList will have as many entries as R and X matrix length
assert(len(graphNodes_noSub)*3==len(R)) # *3 is because R is 3ph
with open("37NF_busList.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='-',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(graphNodes_noSub)

In [4]:
# print z_to_sub for all buses to a csv
graphNodes_nosub = hm.remove_subst_nodes(fin_feeder, file_name) # dont consider substation nodes, node 650 and 651 for 13NF

myabsZs=np.array([], dtype=np.int64).reshape(0,3)

for busName in graphNodes_nosub: # for all non-slack buses
    z2sub=np.around(imp.get_total_impedance_from_substation(fin_feeder, busName,depths),2)
    zself=[[z2sub[0][0], z2sub[1][1], z2sub[2][2]]]
    print('zzelf=',zself)
    abs_z=np.absolute(zself)
    print('abs_z=',abs_z)
    myabsZs=np.append(myabsZs,np.array(abs_z),axis=0) 

print(myabsZs)
#print(np.array(myabsZs))
np.savetxt('37NF_z2sub.csv', np.array(myabsZs), delimiter=',')

zzelf= [[(0.09+0.05j), (0.08+0.05j), (0.09+0.05j)]]
abs_z= [[0.1029563  0.09433981 0.1029563 ]]
zzelf= [[(0.21+0.13j), (0.19+0.12j), (0.21+0.13j)]]
abs_z= [[0.24698178 0.22472205 0.24698178]]
zzelf= [[(0.3+0.17j), (0.3+0.15j), (0.3+0.17j)]]
abs_z= [[0.34481879 0.3354102  0.34481879]]
zzelf= [[(0.25+0.11j), (0.24+0.1j), (0.25+0.11j)]]
abs_z= [[0.27313001 0.26       0.27313001]]
zzelf= [[(0.64+0.34j), (0.64+0.32j), (0.64+0.34j)]]
abs_z= [[0.72470684 0.71554175 0.72470684]]
zzelf= [[(0.86+0.4j), (0.86+0.38j), (0.86+0.4j)]]
abs_z= [[0.94847246 0.94021274 0.94847246]]
zzelf= [[(0.48+0.27j), (0.47+0.25j), (0.48+0.27j)]]
abs_z= [[0.55072679 0.53235327 0.55072679]]
zzelf= [[(0.4+0.23j), (0.39+0.21j), (0.4+0.23j)]]
abs_z= [[0.46141088 0.44294469 0.46141088]]
zzelf= [[(0.9+0.46j), (0.89+0.43j), (0.9+0.46j)]]
abs_z= [[1.0107423 0.9884331 1.0107423]]
zzelf= [[(1.05+0.57j), (1.04+0.53j), (1.05+0.57j)]]
abs_z= [[1.19473847 1.16726175 1.19473847]]
zzelf= [[(0.34+0.15j), (0.34+0.14j), (0.34+0.15j)]]
a